In [2]:
import heapq
from typing import List, Tuple

**57. Insert Interval**


In [ ]:
class Solution:  # 100% time, 33% memory
    """
    Uses binary search
    """

    def insert(
        self, intervals: List[List[int]], newInterval: List[int]
    ) -> List[List[int]]:
        if not intervals:
            return [newInterval]
        # binary search to get the index of insertion
        left, right = 0, len(intervals) - 1
        while left <= right:
            middle = (left + right) // 2
            if intervals[middle][0] < newInterval[0]:
                left = middle + 1
            else:
                right = middle - 1

        # now its a mergeIntervals problem:
        intervals.insert(left, newInterval)

        res = []
        for interval in intervals:
            if not res or res[-1][1] < interval[0]:
                res.append(interval)
            else:
                # since the lower bound will always be smaller in res[-1] than in interval
                # only merge the upper bounds
                res[-1][1] = max(interval[1], res[-1][1])
        return res


class SolutionGreedy:  # 8% time, 33% memory
    """
    Same line of thinking as my mergeIntervals solution
    """

    def insert(
        self, intervals: List[List[int]], newInterval: List[int]
    ) -> List[List[int]]:
        res = []

        for i in range(len(intervals)):
            if newInterval[1] < intervals[i][0]:
                # if the right bound of the newInterval is ever less that the left bound of current
                # we have found the point of no collision and can return because there will be no more overlap
                res.append(newInterval)
                return res + intervals[i:]
            elif newInterval[0] > intervals[i][1]:
                # if the left bound of the newInterval is more than the right bound of current
                # we have found a point of no collision
                res.append(intervals[i])
            else:
                # there is some form of overlap if the strict conditions above are not met
                # merge intervals
                newInterval = [
                    min(newInterval[0], intervals[i][0]),
                    max(newInterval[1], intervals[i][1]),
                ]
        # if the function hasn't returned yet, it is because the newInterval needs to go at the end
        # (since the return condition if it being smaller not larger)
        res.append(newInterval)
        return res


insert = Solution()
print(insert.insert([[1, 3], [6, 9]], [2, 5]))
print(insert.insert([[1, 2], [3, 5], [6, 7], [8, 10], [12, 16]], [4, 8]))
print(insert.insert([], [5, 7]))
print(insert.insert([[1, 5]], [6, 8]))
print(insert.insert([[6, 8]], [1, 5]))

[[1, 5], [6, 9]]
[[1, 2], [3, 10], [12, 16]]
[[5, 7]]
[[1, 5], [6, 8]]
[[1, 5], [6, 8]]


**56. Merge Intervals**

Given an array of intervals where intervals[i] = [starti, endi], merge all overlapping intervals, and return an array of the non-overlapping intervals that cover all the intervals in the input.


In [47]:
class Solution:  # 74% time, 60% memory
    def merge(self, intervals: List[List[int]]) -> List[List[int]]:
        # sorting based on the bigger number in the interval to make my comment inside the while loop true
        intervals.sort(key=lambda x: x[1])
        res = []
        while intervals:
            cur = intervals.pop()

            while intervals and cur[0] <= intervals[-1][1]:
                # the only time there is overlap is when the left bound of an interval is smaller than or equal to right bound of another
                # this is because we are sorting on the right bound, so the right bound of cur will always be the biggest number available
                l, r = intervals.pop()
                cur = [min(cur[0], l), max(cur[1], r)]
            res.append(cur)
        return res[::-1]


S = Solution()
print(S.merge([[1, 3], [2, 6], [8, 10], [15, 18]]))
print(S.merge([[1, 4], [0, 5]]))
print(S.merge([[2, 3], [4, 5], [6, 7], [8, 9], [1, 10]]))

[[1, 6], [8, 10], [15, 18]]
[[0, 5]]
[[1, 10]]


**435. Non-overlapping arrays**

I solved this in O(nlog(n) + n) in Go (`NeetCode/go/intervals.go`)

-   Sorted intervals by start. O(nlogn)
-   Greedily removed intervals that overlapped. O(n)


In [ ]:
class Solution:
    def eraseOverlapIntervals(self, intervals: List[List[int]]) -> int:
        """
        This is neetcode's bottom up DP solution O(n^2) because every pair (i,j) j < i is tested.
        For each index you get the maximum number of non-overlapping intervals you can form by including the interval at the index

        Then just return the largest number out of those
        """
        intervals.sort(key=lambda x: x[1])
        n = len(intervals)
        dp = [0] * n

        for i in range(n):
            dp[i] = (
                1  # every index will have at least 1 non-overlapping interval if it is included
            )
            # (itself with all others removed)

            for j in range(i):
                # go through all arrays whose interval end are smaller than index i's end
                if intervals[j][1] <= intervals[i][0]:
                    # if no overlap, update the value stored in dp
                    dp[i] = max(dp[i], 1 + dp[j])

        max_non_overlapping = max(dp)
        return n - max_non_overlapping

    def eraseOverlapIntervalsBinarySearch(self, intervals: List[List[int]]) -> int:
        """
        Ask copilot to explain this

        O(nlogn) time
        O(n) space
        """
        intervals.sort(key=lambda x: x[1])
        n = len(intervals)
        dp = [0] * n
        dp[0] = 1

        def bs(r, target):
            l = 0
            while l < r:
                m = (l + r) >> 1
                if intervals[m][1] <= target:
                    l = m + 1
                else:
                    r = m
            return l

        for i in range(1, n):
            idx = bs(i, intervals[i][0])
            if idx == 0:
                dp[i] = dp[i - 1]
            else:
                dp[i] = max(dp[i - 1], 1 + dp[idx - 1])
        return n - dp[n - 1]

**Meeting Rooms**


In [ ]:
class Interval(object):
    def __init__(self, start, end):
        self.start = start
        self.end = end


class Solution:
    def canAttendMeetings(self, intervals: List[Interval]) -> bool:
        """
        if a meeting ends at 8 and a new one starts at 8 that is okay. Overlap has to be actual overlap
        """
        intervals.sort(key=lambda ivl: ivl.start)  # sort by start times
        for i in range(1, len(intervals)):
            if intervals[i].start < intervals[i - 1].end:
                return False
        return True

**Meeting Rooms II**


In [12]:
class Interval(object):
    def __init__(self, start, end):
        self.start = start
        self.end = end


class Solution:
    def minMeetingRooms(self, intervals: List[Interval]) -> int:
        """
        My thinking is I can reuse the canAttendMeetings method with a small modification
        Whenever it fails, the day that ends the latest of the two days gets sent to the next day
            I actualy don't think I need to care about which ends later
            [[0,8], [1,6], [3,9]] -> 3 days which you get just by kicking out the one causing the issue
            [[0,8], [1,6], [6,9]] -> 2 days which you get just by kicking out the one causing the issue
            [[0,4], [1,6], [4,7], [6,8]] -> 2 days which you get just by kicking out the one causing the issue
        Note that the check only needs to occur with the last interval that was processed in the day since all days will be sorted by start and will be valid.
        Need to keep track of the effective end hour for each date. Don't need to store any actual intervals
        """
        intervals.sort(key=lambda ivl: ivl.start)  # sort by start times
        endOfDays = []
        for i in range(len(intervals)):
            needNewDay = True
            for j in range(len(endOfDays)):
                if intervals[i].start >= endOfDays[j]:
                    endOfDays[j] = intervals[i].end
                    needNewDay = False
                    break

            if needNewDay:
                endOfDays.append(intervals[i].end)
        return len(endOfDays)


minMeetingRooms = Solution()
print(minMeetingRooms.minMeetingRooms([]))
print(
    minMeetingRooms.minMeetingRooms(
        [Interval(0, 40), Interval(5, 10), Interval(15, 20)]
    )
)
print(minMeetingRooms.minMeetingRooms([Interval(0, 8), Interval(1, 6), Interval(3, 9)]))
print(minMeetingRooms.minMeetingRooms([Interval(0, 8), Interval(1, 6), Interval(6, 9)]))

0
2
3
2


**1851. Minimum Interval to Include Each Query**

[Visualgo - Binary Segment Tree with Lazy Propagation](https://visualgo.net/en/segmenttree)


In [12]:
class Solution:
    """
    A faster runtime solution is to use a dictionary mapping each query to its index
    """

    # 28% time, 35% memory (when I use sorted(zip) to keep track of the query indices)
    # 83% time, 94% memory (when I make sorted_q_idx like this)
    def minInterval(self, intervals: List[List[int]], queries: List[int]) -> List[int]:
        # sort by the start of the interval
        n = len(intervals)
        m = len(queries)
        intervals.sort()
        # sort the queries too, because it'll make using a heap possible
        sorted_q_idx = sorted(list(range(m)), key=lambda i: queries[i])

        minHeap = []  # (size, interval end)
        interval_idx = 0

        result = [-1] * m
        for idx in sorted_q_idx:
            query = queries[idx]
            while interval_idx < n and intervals[interval_idx][0] <= query:
                # populate heap with new interval candidates
                # (only need to add the end of the interval to the heap because of the if-statement)
                start, end = intervals[interval_idx]
                heapq.heappush(minHeap, (end - start + 1, end))
                interval_idx += 1

            while minHeap and minHeap[0][1] < query:
                # clean up the heap by remove unusable intervals
                # unusable if the end of the interval is before the query
                heapq.heappop(minHeap)

            if minHeap:
                # if there is an interval to use:
                # size, _, _ = minHeap[0]
                result[idx] = minHeap[0][0]

        return result


class SegmentTree:
    def __init__(self, N):
        self.n = N
        self.tree = [float("inf")] * (4 * N)
        self.lazy = [float("inf")] * (4 * N)

    def propagate(self, treeidx, lo, hi):
        if self.lazy[treeidx] != float("inf"):
            self.tree[treeidx] = min(self.tree[treeidx], self.lazy[treeidx])
            if lo != hi:
                self.lazy[2 * treeidx + 1] = min(
                    self.lazy[2 * treeidx + 1], self.lazy[treeidx]
                )
                self.lazy[2 * treeidx + 2] = min(
                    self.lazy[2 * treeidx + 2], self.lazy[treeidx]
                )
            self.lazy[treeidx] = float("inf")

    def update(self, treeidx, lo, hi, left, right, val):
        self.propagate(treeidx, lo, hi)
        if lo > right or hi < left:
            return
        if lo >= left and hi <= right:
            self.lazy[treeidx] = min(self.lazy[treeidx], val)
            self.propagate(treeidx, lo, hi)
            return
        mid = (lo + hi) // 2
        self.update(2 * treeidx + 1, lo, mid, left, right, val)
        self.update(2 * treeidx + 2, mid + 1, hi, left, right, val)
        self.tree[treeidx] = min(self.tree[2 * treeidx + 1], self.tree[2 * treeidx + 2])

    def query(self, treeidx, lo, hi, idx):
        self.propagate(treeidx, lo, hi)
        if lo == hi:
            return self.tree[treeidx]
        mid = (lo + hi) // 2
        if idx <= mid:
            return self.query(2 * treeidx + 1, lo, mid, idx)
        else:
            return self.query(2 * treeidx + 2, mid + 1, hi, idx)

    def update_range(self, left, right, val):
        self.update(0, 0, self.n - 1, left, right, val)

    def query_point(self, idx):
        return self.query(0, 0, self.n - 1, idx)


class SolutionWithSegmentTree:
    def minInterval(self, intervals: List[List[int]], queries: List[int]) -> List[int]:
        points = []
        for interval in intervals:
            points.append(interval[0])
            points.append(interval[1])
        for q in queries:
            points.append(q)

        # Compress the coordinates
        points = sorted(set(points))
        compress = {points[i]: i for i in range(len(points))}

        # Lazy Segment Tree
        segTree = SegmentTree(len(points))

        for interval in intervals:
            start = compress[interval[0]]
            end = compress[interval[1]]
            length = interval[1] - interval[0] + 1
            segTree.update_range(start, end, length)

        ans = []
        for q in queries:
            idx = compress[q]

            # query for minSize
            res = segTree.query_point(idx)
            ans.append(res if res != float("inf") else -1)
        return ans


minInterval = Solution()
minInterval.minInterval([[1, 4], [2, 4], [0, 6], [4, 4]], [2, 5, 4, 3])

[3, 7, 1, 3]

In [13]:
import numpy as np


def do_intersect_matrix(P1, P2, P3, P4):
    # Extract coordinates
    x1, y1 = P1
    x2, y2 = P2
    x3, y3 = P3
    x4, y4 = P4

    # Determinant of the coefficient matrix
    determinant = (x2 - x1) * (y3 - y4) - (x3 - x4) * (y2 - y1)

    if determinant == 0:
        # Lines are parallel or coincident
        if (lambda x, y: ((y - y3) * (x4 - x3) - (x - x3) * (y4 - y3)))(x1, y1) == 0:
            # if plugging in a point from Line 1 into the parametric equation for Line 2 results in that Zero
            # lines are coincident
            return True
        return False

    # Numerators for t and u
    t_numerator = (x3 - x1) * (y3 - y4) - (x3 - x4) * (y3 - y1)
    u_numerator = (x2 - x1) * (y3 - y1) - (x3 - x1) * (y2 - y1)

    # Calculate t and u
    t = t_numerator / determinant
    u = u_numerator / determinant

    # Check if t and u are within [0, 1]
    return 0 <= t <= 1 and 0 <= u <= 1


P1 = np.array([1, 1])
P2 = np.array([4, 4])
P3 = np.array([2, 1])
P4 = np.array([3, 8])

do_intersect_matrix(P1, P2, P3, P4)

P1 = np.array([3, 0])
P2 = np.array([6, 3])
P3 = np.array([2, -1])
P4 = np.array([5, 2])

do_intersect_matrix(P1, P2, P3, P4)

-18
[2.16666667 2.16666667] [2.16666667 2.16666667]
0


True

**3169. Count Days Without Meetings**


In [ ]:
class Solution:
    # 14% time, 13% memory
    def countDays(self, days: int, meetings: List[List[int]]) -> int:
        """
        Line Sweep pre sure
        """
        events = sorted(
            [(meeting[0], 0) for meeting in meetings]
            + [(meeting[1], 1) for meeting in meetings]
        )
        day = 0
        active = 0
        result = 0
        for e, T in events:
            if not active:
                # can only be tru if T == 0 (start event)
                # events are inclusive so we need to subtract one
                result += max(0, e - day - 1)

            if T:  # end
                active -= 1
            else:
                active += 1
            day = e

            if day >= days:
                break

        result += max(0, days - day)
        return result

    def countDaysFaster(self, days: int, meetings: List[List[int]]) -> int:
        """Better solution"""

        meetings = sorted(meetings)
        free_days = 0
        latest_end = 0

        for start, end in meetings:
            if start > latest_end + 1:
                free_days += start - latest_end - 1
            latest_end = max(latest_end, end)

        free_days += days - latest_end

        return free_days


countDays = Solution()
print(countDays.countDays(10, [[5, 7], [1, 3], [9, 10]]))
print(countDays.countDays(5, [[2, 4], [1, 3]]))
print(countDays.countDays(6, [[1, 6]]))

2
1
0


**3394. Check if Grid can be Cut into Sections**


In [ ]:
class Solution:
    # 84% time, 70% memory (as written)
    # 72% time, 99% memory (if I use `xs` and `ys`)
    def checkValidCuts(self, n: int, rectangles: List[List[int]]) -> bool:
        """
        2 line sweeps
        - one for horizontal scan (vertical lines) sorted by start_x
        - one for vertical scan (horizontal lines) sorted by start_y
        """

        # xs = sorted([(x[0], x[2]) for x in rectangles])
        # ys = sorted([(y[1], y[3]) for y in rectangles])

        def canCut(points: List[Tuple[int, int]]) -> bool:
            count = 0
            maxEnd = points[0][1]
            for s, e in points:
                if s >= maxEnd:
                    # we have reached a point where there are no active rectangles we can add a cut
                    count += 1
                    if count == 2:
                        return True
                maxEnd = max(e, maxEnd)
            return False

        return canCut(sorted([(x[0], x[2]) for x in rectangles])) or canCut(
            sorted([(y[1], y[3]) for y in rectangles])
        )


checkValidCuts = Solution()
print(
    checkValidCuts.checkValidCuts(
        5, [[1, 0, 5, 2], [0, 2, 2, 4], [3, 2, 5, 3], [0, 4, 4, 5]]
    )
)
print(
    checkValidCuts.checkValidCuts(
        4, [[0, 0, 1, 1], [2, 0, 3, 4], [0, 2, 2, 3], [3, 0, 4, 3]]
    )
)
print(
    checkValidCuts.checkValidCuts(
        4, [[0, 2, 2, 4], [1, 0, 3, 2], [2, 2, 3, 4], [3, 0, 4, 2], [3, 2, 4, 4]]
    )
)

True
True
False
